In [2]:
%%writefile config.yaml
models:
  # - model: malhajar/Llama-2-7b-chat-tr # Turkish
  - model: Trendyol/Trendyol-LLM-7b-chat-v0.1
    parameters:
      density: [0.5, 0.5, 0.5] # density gradient
      weight: 0.5
  - model: swap-uniba/LLaMAntino-2-chat-7b-hf-UltraChat-ITA # Italian
    parameters:
      density: 0.5
      weight: [0.5, 0.5, 0.5, 0.5] # weight gradient
merge_method: ties
base_model: NousResearch/Llama-2-7b-chat-hf
parameters:
  normalize: true
  int8_mask: true
dtype: float16


Overwriting config.yaml


In [3]:
%%writefile config.yaml
models:
  # - model: malhajar/Llama-2-7b-chat-tr # Turkish
  - model: Trendyol/Trendyol-LLM-7b-chat-v0.1
  - model: swap-uniba/LLaMAntino-2-chat-7b-hf-UltraChat-ITA # Italian
merge_method: slerp
# base_model: psmathur/orca_mini_v3_13b
# base_model: malhajar/Llama-2-7b-chat-tr # Turkish
base_model: Trendyol/Trendyol-LLM-7b-chat-v0.1
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5 # fallback for rest of tensors
dtype: float16

Overwriting config.yaml


In [ ]:
!mergekit-yaml config.yaml ./merged_model --cuda

/home/adam/miniconda3/envs/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Executing graph: 100%|██████████████████████| 1457/1457 [01:20<00:00, 18.20it/s]


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForCausalLM, LlamaTokenizer, pipeline

model_path = "merged_model"  # Change this to your actual model path

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test the model
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"])

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

In [6]:

sampling_params = dict(do_sample=True, temperature=0.3, top_k=50, top_p=0.9)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto",
                max_new_tokens=64,
                return_full_text=True,
                repetition_penalty=1.1
               )

DEFAULT_SYSTEM_PROMPT = "Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın.\n"

TEMPLATE = (
    "[INST] <<SYS>>\n"
    "{system_prompt}\n"
    "<</SYS>>\n\n"
    "{instruction} [/INST]"
)

def generate_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
    return TEMPLATE.format_map({'instruction': instruction,'system_prompt': system_prompt})

def generate_output(user_query, sys_prompt=DEFAULT_SYSTEM_PROMPT):
    prompt = generate_prompt(user_query, sys_prompt)
    outputs = pipe(prompt,
               **sampling_params
              )
    return outputs[0]["generated_text"].split("[/INST]")[-1]

instructions = """Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın. Görevin sana verilen cümleleri ingilizce diline tercüme etmek. Birkaç tane örnek:
1.	Haberler onu üzdü.	-	La notizia l'ha resa triste.
2.	Çok şarap içmiyorum.	-	Non bevo molto vino.
3.	Nancy onu bir partiye çağırdı.	-	Nancy lo invitò ad una festa.
4.	Çok fazla yersen şişmanlarsın.	-	Mangiando troppo si ingrassa.
Bundan sonra sana verilen her cümleyi italyanca diline tercüme et:"""

user_query = "Benim bir evim var."

response = generate_output(f"{instructions} {user_query}")
print(response)

NameError: name 'model' is not defined